In [1]:
import pandas as pd
import string
import nltk
import math
import tensorflow as tfl
import numpy as np
import operator
import regex as re
import os

from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.tag import UnigramTagger
from nltk.corpus import treebank
from nltk.corpus import sentiwordnet as swn
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

pd.set_option('display.max_columns', None)

In [2]:
file = pd.read_csv("scrapping_BTC_comments.csv")
file

,created_utc,body,score,permalink,date
0,1.511280e+09,So /u/spez how much longer are you going to ig...,2290,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:06:12
1,1.511281e+09,Wow this is huge. I'm surprised /r/bitcoin mod...,631,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:11:11
2,1.511278e+09,Holy shit....,720,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 16:32:00
3,1.511279e+09,"I'm a bot, *bleep*, *bloop*. Someone has linke...",244,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 16:46:08
4,1.511291e+09,/u/spez I once shied away from replying to you...,34,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 20:00:57
5,1.511280e+09,I don't know whether I should feel enraged by ...,212,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:06:14
6,1.511280e+09,Great detective work!\r\n\r\n$10 /u/tippr,127,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:03:40
7,1.511281e+09,"Wow, epic, standing ovation! This confirms wha...",164,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:17:23
8,1.511289e+09,This comment from /u/4n4n4 got the double dash...,23,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 19:37:51
9,1.511289e+09,If I remember correctly Gregory Maxwell often ...,115,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 19:29:34


In [3]:
len_file = len(file)
len_file

167

## Data Cleaning

In [4]:
file["body"]

0      So /u/spez how much longer are you going to ig...
1      Wow this is huge. I'm surprised /r/bitcoin mod...
2                                          Holy shit....
3      I'm a bot, *bleep*, *bloop*. Someone has linke...
4      /u/spez I once shied away from replying to you...
5      I don't know whether I should feel enraged by ...
6              Great detective work!\r\n\r\n$10 /u/tippr
7      Wow, epic, standing ovation! This confirms wha...
8      This comment from /u/4n4n4 got the double dash...
9      If I remember correctly Gregory Maxwell often ...
10     Paging u/adam3us, what do you feel about a C-l...
11     Smoke everywhere.  /u/bashco clearly had knowl...
12     The technical term is "false flag".\r\n\r\n>  ...
13                                               TL;DR? 
14                       Initiate Operation Dragonslayer
15     Thank you for doing this. More evidence to col...
16     The most telling aspect of this is the solid l...
17     I don't know who this ce

### Set Q1

In [5]:
scores = file["score"].values
scores = np.sort(scores) 
scores

array([   2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
          2,    2,    3,    3,    3,    3,    3,    3,    3,    3,    3,
          3,    3,    3,    3,    3,    3,    3,    3,    3,    4,    4,
          4,    4,    4,    4,    4,    4,    5,    5,    5,    5,    5,
          5,    5,    5,    5,    6,    6,    6,    6,    6,    6,    6,
          6,    6,    6,    6,    6,    7,    7,    7,    7,    7,    7,
          7,    7,    7,    7,    7,    8,    8,    8,    8,    8,    8,
          8,    8,    8,    8,    8,    9,    9,    9,    9,    9,    9,
          9,    9,    9,    9,    9,    9,   10,   10,   10,   10,   10,
         10,   11,   11,   11,   11,   11,   11,   12,   12,   12,   12,
         12,   12,   12,   12,   12,   13,   13,   13,   13,   14,   14,
         16,   16,   16,   16,   17,   18,   18,   19,   19,   19,   19,
         20,   20,   20,   21,   21,   22,   22,   23,   28,   29,   34,
         34,   35,   43,   47,   48,   49,   52,   

In [6]:
indexQ1 = int(len_file/4)
indexQ1
Q1 = scores[indexQ1]
Q1

5

### Remove comments whose score < Q1

In [7]:
file['score'][0]

2290

In [8]:
for i in range(len_file):
    if file['score'][i] < Q1:
        file = file.drop(i)
        
file

,created_utc,body,score,permalink,date
0,1.511280e+09,So /u/spez how much longer are you going to ig...,2290,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:06:12
1,1.511281e+09,Wow this is huge. I'm surprised /r/bitcoin mod...,631,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:11:11
2,1.511278e+09,Holy shit....,720,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 16:32:00
3,1.511279e+09,"I'm a bot, *bleep*, *bloop*. Someone has linke...",244,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 16:46:08
4,1.511291e+09,/u/spez I once shied away from replying to you...,34,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 20:00:57
5,1.511280e+09,I don't know whether I should feel enraged by ...,212,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:06:14
6,1.511280e+09,Great detective work!\r\n\r\n$10 /u/tippr,127,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:03:40
7,1.511281e+09,"Wow, epic, standing ovation! This confirms wha...",164,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:17:23
8,1.511289e+09,This comment from /u/4n4n4 got the double dash...,23,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 19:37:51
9,1.511289e+09,If I remember correctly Gregory Maxwell often ...,115,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 19:29:34


### Remove all "/u/", "/r/", "/r", "\n"

In [10]:
for i in range(len_file):
    try:
        file["body"][i] = re.sub(r"(\/\w\/|\w\/)", "", file["body"][i])
        file["body"][i] = re.sub(r"\\n?t?", "", file["body"][i])
        file["body"][i] = re.sub(r"[0-9]*", "", file["body"][i])
    except KeyError :
        continue

file.head(20)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


,created_utc,body,score,permalink,date
0,1.511280e+09,So spez how much longer are you going to ignor...,2290,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:06:12
1,1.511281e+09,Wow this is huge. I'm surprised bitcoin mods w...,631,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:11:11
2,1.511278e+09,Holy shit....,720,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 16:32:00
3,1.511279e+09,"I'm a bot, *bleep*, *bloop*. Someone has linke...",244,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 16:46:08
4,1.511291e+09,spez I once shied away from replying to you re...,34,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 20:00:57
5,1.511280e+09,I don't know whether I should feel enraged by ...,212,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:06:14
6,1.511280e+09,Great detective work!\r\n\r\n$ tippr,127,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:03:40
7,1.511281e+09,"Wow, epic, standing ovation! This confirms wha...",164,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:17:23
8,1.511289e+09,"This comment from nn got the double dash, no s...",23,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 19:37:51
9,1.511289e+09,If I remember correctly Gregory Maxwell often ...,115,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 19:29:34


### Remove punctuation

In [11]:
def remove_punctuation(text):
    no_punct = "".join([c for c in text if c not in string.punctuation])
    return no_punct

In [12]:
file['body'] = file['body'].apply(lambda x: remove_punctuation(x))
file

,created_utc,body,score,permalink,date
0,1.511280e+09,So spez how much longer are you going to ignor...,2290,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:06:12
1,1.511281e+09,Wow this is huge Im surprised bitcoin mods wer...,631,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:11:11
2,1.511278e+09,Holy shit,720,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 16:32:00
3,1.511279e+09,Im a bot bleep bloop Someone has linked to thi...,244,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 16:46:08
4,1.511291e+09,spez I once shied away from replying to you re...,34,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 20:00:57
5,1.511280e+09,I dont know whether I should feel enraged by w...,212,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:06:14
6,1.511280e+09,Great detective work\r\n\r\n tippr,127,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:03:40
7,1.511281e+09,Wow epic standing ovation This confirms what w...,164,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:17:23
8,1.511289e+09,This comment from nn got the double dash no sp...,23,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 19:37:51
9,1.511289e+09,If I remember correctly Gregory Maxwell often ...,115,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 19:29:34


### Remove short comments (in terms of words)

In [13]:
tokenizer = RegexpTokenizer(r'\w+')

In [14]:
file['body'] = file['body'].apply(lambda x : tokenizer.tokenize(x.lower()))
file.head(20)

,created_utc,body,score,permalink,date
0,1.511280e+09,"[so, spez, how, much, longer, are, you, going,...",2290,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:06:12
1,1.511281e+09,"[wow, this, is, huge, im, surprised, bitcoin, ...",631,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:11:11
2,1.511278e+09,"[holy, shit]",720,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 16:32:00
3,1.511279e+09,"[im, a, bot, bleep, bloop, someone, has, linke...",244,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 16:46:08
4,1.511291e+09,"[spez, i, once, shied, away, from, replying, t...",34,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 20:00:57
5,1.511280e+09,"[i, dont, know, whether, i, should, feel, enra...",212,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:06:14
6,1.511280e+09,"[great, detective, work, tippr]",127,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:03:40
7,1.511281e+09,"[wow, epic, standing, ovation, this, confirms,...",164,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:17:23
8,1.511289e+09,"[this, comment, from, nn, got, the, double, da...",23,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 19:37:51
9,1.511289e+09,"[if, i, remember, correctly, gregory, maxwell,...",115,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 19:29:34


In [15]:
file = file[file['body'].map(len) > 2]
file

,created_utc,body,score,permalink,date
0,1.511280e+09,"[so, spez, how, much, longer, are, you, going,...",2290,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:06:12
1,1.511281e+09,"[wow, this, is, huge, im, surprised, bitcoin, ...",631,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:11:11
3,1.511279e+09,"[im, a, bot, bleep, bloop, someone, has, linke...",244,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 16:46:08
4,1.511291e+09,"[spez, i, once, shied, away, from, replying, t...",34,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 20:00:57
5,1.511280e+09,"[i, dont, know, whether, i, should, feel, enra...",212,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:06:14
6,1.511280e+09,"[great, detective, work, tippr]",127,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:03:40
7,1.511281e+09,"[wow, epic, standing, ovation, this, confirms,...",164,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:17:23
8,1.511289e+09,"[this, comment, from, nn, got, the, double, da...",23,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 19:37:51
9,1.511289e+09,"[if, i, remember, correctly, gregory, maxwell,...",115,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 19:29:34
10,1.511297e+09,"[paging, adamus, what, do, you, feel, about, a...",22,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 21:46:16


### Remove small words

In [16]:
for i in range(len_file):
    try :
        for word in file['body'][i]:
            if len(word) < 3:
                file['body'][i].remove(word)
    except KeyError :
        continue
        
file

,created_utc,body,score,permalink,date
0,1.511280e+09,"[spez, how, much, longer, are, you, going, ign...",2290,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:06:12
1,1.511281e+09,"[wow, this, huge, surprised, bitcoin, mods, we...",631,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:11:11
3,1.511279e+09,"[a, bot, bleep, bloop, someone, has, linked, t...",244,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 16:46:08
4,1.511291e+09,"[spez, once, shied, away, from, replying, you,...",34,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 20:00:57
5,1.511280e+09,"[dont, know, whether, should, feel, enraged, w...",212,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:06:14
6,1.511280e+09,"[great, detective, work, tippr]",127,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:03:40
7,1.511281e+09,"[wow, epic, standing, ovation, this, confirms,...",164,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:17:23
8,1.511289e+09,"[this, comment, from, got, the, double, dash, ...",23,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 19:37:51
9,1.511289e+09,"[remember, correctly, gregory, maxwell, often,...",115,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 19:29:34
10,1.511297e+09,"[paging, adamus, what, you, feel, about, cleve...",22,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 21:46:16


### Remove Stopwords

In [17]:
def remove_stopwords(text):
    words = [w for w in text if w not in stopwords.words('english')]
    return words

In [18]:
file['body'] = file['body'].apply(lambda x : remove_stopwords(x))
file

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,created_utc,body,score,permalink,date
0,1.511280e+09,"[spez, much, longer, going, ignore, blatant, c...",2290,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:06:12
1,1.511281e+09,"[wow, huge, surprised, bitcoin, mods, sloppy, ...",631,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:11:11
3,1.511279e+09,"[bot, bleep, bloop, someone, linked, thread, a...",244,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 16:46:08
4,1.511291e+09,"[spez, shied, away, replying, reddit, admins, ...",34,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 20:00:57
5,1.511280e+09,"[dont, know, whether, feel, enraged, happened,...",212,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:06:14
6,1.511280e+09,"[great, detective, work, tippr]",127,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:03:40
7,1.511281e+09,"[wow, epic, standing, ovation, confirms, suspe...",164,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:17:23
8,1.511289e+09,"[comment, got, double, dash, space, talk, usin...",23,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 19:37:51
9,1.511289e+09,"[remember, correctly, gregory, maxwell, often,...",115,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 19:29:34
10,1.511297e+09,"[paging, adamus, feel, clevel, executive, comp...",22,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 21:46:16


### Remove Reddit links

In [19]:
def remove_links(text):
    words = [w for w in text if not len(w) > 20]
    return words

In [20]:
file['body'] = file['body'].apply(lambda x : remove_links(x))
file

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,created_utc,body,score,permalink,date
0,1.511280e+09,"[spez, much, longer, going, ignore, blatant, c...",2290,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:06:12
1,1.511281e+09,"[wow, huge, surprised, bitcoin, mods, sloppy, ...",631,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:11:11
3,1.511279e+09,"[bot, bleep, bloop, someone, linked, thread, a...",244,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 16:46:08
4,1.511291e+09,"[spez, shied, away, replying, reddit, admins, ...",34,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 20:00:57
5,1.511280e+09,"[dont, know, whether, feel, enraged, happened,...",212,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:06:14
6,1.511280e+09,"[great, detective, work, tippr]",127,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:03:40
7,1.511281e+09,"[wow, epic, standing, ovation, confirms, suspe...",164,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:17:23
8,1.511289e+09,"[comment, got, double, dash, space, talk, usin...",23,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 19:37:51
9,1.511289e+09,"[remember, correctly, gregory, maxwell, often,...",115,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 19:29:34
10,1.511297e+09,"[paging, adamus, feel, clevel, executive, comp...",22,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 21:46:16


### Lemmatization

In [21]:
train_sents = treebank.tagged_sents()[:3000]
tagger = UnigramTagger(train_sents)

In [22]:
def lemmatize(text):
    lemmatizer = WordNetLemmatizer()
    lemmatizedText = []
    words = nltk.word_tokenize(text)
    wordTokens = nltk.pos_tag(words)
    for wordToken in wordTokens:
        wordType = wordToken[1]
        word =  wordToken[0]
        
        
        if wordType[0:2] == "JJ":
            lemmatizedText.append(lemmatizer.lemmatize(word, "a"))
        elif wordType[0] == "V":
            lemmatizedText.append(lemmatizer.lemmatize(word, "v"))
        elif wordType[0] == "N":
            lemmatizedText.append(lemmatizer.lemmatize(word, "n"))
        elif wordType[0:2] == "RB":
            lemmatizedText.append(lemmatizer.lemmatize(word, "r"))
        else :
            lemmatizedText.append(lemmatizer.lemmatize(word))            
            
    return " ".join(lemmatizedText)

In [23]:
for i in range(len_file):
    try :
        for word in file["body"][i]:
            word = lemmatize(word)
    except KeyError :
        continue
        
file

,created_utc,body,score,permalink,date
0,1.511280e+09,"[spez, much, longer, going, ignore, blatant, c...",2290,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:06:12
1,1.511281e+09,"[wow, huge, surprised, bitcoin, mods, sloppy, ...",631,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:11:11
3,1.511279e+09,"[bot, bleep, bloop, someone, linked, thread, a...",244,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 16:46:08
4,1.511291e+09,"[spez, shied, away, replying, reddit, admins, ...",34,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 20:00:57
5,1.511280e+09,"[dont, know, whether, feel, enraged, happened,...",212,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:06:14
6,1.511280e+09,"[great, detective, work, tippr]",127,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:03:40
7,1.511281e+09,"[wow, epic, standing, ovation, confirms, suspe...",164,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:17:23
8,1.511289e+09,"[comment, got, double, dash, space, talk, usin...",23,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 19:37:51
9,1.511289e+09,"[remember, correctly, gregory, maxwell, often,...",115,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 19:29:34
10,1.511297e+09,"[paging, adamus, feel, clevel, executive, comp...",22,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 21:46:16


### Stemming

In [24]:
porter = PorterStemmer()

In [25]:
def stem_sentence(sentence):
    porter = PorterStemmer()
    for i in range(len(sentence)):
        word = porter.stem(sentence[i])
        sentence[i] = word
    return sentence

In [26]:
for i in range(len_file):
    try:
        file["body"][i] = stem_sentence(file["body"][i])
    except KeyError:
        continue
file

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
D:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3325: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


,created_utc,body,score,permalink,date
0,1.511280e+09,"[spez, much, longer, go, ignor, blatant, censo...",2290,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:06:12
1,1.511281e+09,"[wow, huge, surpris, bitcoin, mod, sloppi, abl...",631,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:11:11
3,1.511279e+09,"[bot, bleep, bloop, someon, link, thread, anot...",244,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 16:46:08
4,1.511291e+09,"[spez, shi, away, repli, reddit, admin, ask, p...",34,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 20:00:57
5,1.511280e+09,"[dont, know, whether, feel, enrag, happen, ast...",212,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:06:14
6,1.511280e+09,"[great, detect, work, tippr]",127,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:03:40
7,1.511281e+09,"[wow, epic, stand, ovat, confirm, suspect, alo...",164,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:17:23
8,1.511289e+09,"[comment, got, doubl, dash, space, talk, use, ...",23,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 19:37:51
9,1.511289e+09,"[rememb, correctli, gregori, maxwel, often, re...",115,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 19:29:34
10,1.511297e+09,"[page, adamu, feel, clevel, execut, compani, m...",22,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 21:46:16


## Data analysis

### Unique words

In [27]:
uniqueWords = []

for sentence in file["body"]:
    words = [word for word in sentence]
    for word in words:
        if word not in uniqueWords:
            uniqueWords.append(word)

uniqueWords

['spez',
 'much',
 'longer',
 'go',
 'ignor',
 'blatant',
 'censorship',
 'manipul',
 'bitcoin',
 'mod',
 'wow',
 'huge',
 'surpris',
 'sloppi',
 'abl',
 'uncov',
 'vote',
 'bot',
 'mean',
 'involv',
 'hack',
 'wonder',
 'actual',
 'what',
 'crazi',
 'connect',
 'nullc',
 'show',
 'clearli',
 'blockstream',
 'work',
 'togeth',
 'entir',
 'market',
 'dumbfound',
 'would',
 'realli',
 'dont',
 'see',
 'reddit',
 'admin',
 'take',
 'action',
 'user',
 'account',
 'edit',
 'highli',
 'recommend',
 'messag',
 'look',
 'bleep',
 'bloop',
 'someon',
 'link',
 'thread',
 'anoth',
 'place',
 'againstastroturf',
 'evid',
 'may',
 'attack',
 'bcore',
 'berniesand',
 'bgold',
 'inform',
 'bitcoinexpos',
 'bitcoinuncensor',
 'bitcoincash',
 'bitcoindiscuss',
 'gmaxwel',
 'bitcoinmtl',
 'bitcoinno',
 'bitcoinxt',
 'cto',
 'greg',
 'maxwel',
 'caught',
 'use',
 'sock',
 'puppet',
 'scheme',
 'btcgreec',
 'buttcoin',
 'butteri',
 'casualconvers',
 'conspiraci',
 'crypto',
 'cryptocurr',
 'cryptomarket

In [28]:
numOfWords = dict.fromkeys(uniqueWords, 0)
numOfWords

{'spez': 0,
 'much': 0,
 'longer': 0,
 'go': 0,
 'ignor': 0,
 'blatant': 0,
 'censorship': 0,
 'manipul': 0,
 'bitcoin': 0,
 'mod': 0,
 'wow': 0,
 'huge': 0,
 'surpris': 0,
 'sloppi': 0,
 'abl': 0,
 'uncov': 0,
 'vote': 0,
 'bot': 0,
 'mean': 0,
 'involv': 0,
 'hack': 0,
 'wonder': 0,
 'actual': 0,
 'what': 0,
 'crazi': 0,
 'connect': 0,
 'nullc': 0,
 'show': 0,
 'clearli': 0,
 'blockstream': 0,
 'work': 0,
 'togeth': 0,
 'entir': 0,
 'market': 0,
 'dumbfound': 0,
 'would': 0,
 'realli': 0,
 'dont': 0,
 'see': 0,
 'reddit': 0,
 'admin': 0,
 'take': 0,
 'action': 0,
 'user': 0,
 'account': 0,
 'edit': 0,
 'highli': 0,
 'recommend': 0,
 'messag': 0,
 'look': 0,
 'bleep': 0,
 'bloop': 0,
 'someon': 0,
 'link': 0,
 'thread': 0,
 'anoth': 0,
 'place': 0,
 'againstastroturf': 0,
 'evid': 0,
 'may': 0,
 'attack': 0,
 'bcore': 0,
 'berniesand': 0,
 'bgold': 0,
 'inform': 0,
 'bitcoinexpos': 0,
 'bitcoinuncensor': 0,
 'bitcoincash': 0,
 'bitcoindiscuss': 0,
 'gmaxwel': 0,
 'bitcoinmtl': 0,
 'bi

In [29]:
for i in range(len_file):
    try :
        for word in file["body"][i]:
            numOfWords[word] += 1
    except KeyError:
        i += 1
        
numOfWords

{'spez': 4,
 'much': 9,
 'longer': 2,
 'go': 11,
 'ignor': 1,
 'blatant': 2,
 'censorship': 10,
 'manipul': 47,
 'bitcoin': 103,
 'mod': 46,
 'wow': 4,
 'huge': 1,
 'surpris': 8,
 'sloppi': 1,
 'abl': 4,
 'uncov': 2,
 'vote': 47,
 'bot': 18,
 'mean': 3,
 'involv': 37,
 'hack': 39,
 'wonder': 2,
 'actual': 6,
 'what': 4,
 'crazi': 1,
 'connect': 1,
 'nullc': 11,
 'show': 4,
 'clearli': 4,
 'blockstream': 15,
 'work': 20,
 'togeth': 2,
 'entir': 4,
 'market': 2,
 'dumbfound': 1,
 'would': 15,
 'realli': 6,
 'dont': 10,
 'see': 13,
 'reddit': 20,
 'admin': 8,
 'take': 8,
 'action': 2,
 'user': 3,
 'account': 8,
 'edit': 2,
 'highli': 3,
 'recommend': 1,
 'messag': 1,
 'look': 8,
 'bleep': 1,
 'bloop': 1,
 'someon': 7,
 'link': 6,
 'thread': 10,
 'anoth': 4,
 'place': 2,
 'againstastroturf': 1,
 'evid': 37,
 'may': 30,
 'attack': 43,
 'bcore': 1,
 'berniesand': 1,
 'bgold': 1,
 'inform': 4,
 'bitcoinexpos': 1,
 'bitcoinuncensor': 1,
 'bitcoincash': 1,
 'bitcoindiscuss': 1,
 'gmaxwel': 1,
 

### Term Frequency (TF)

In [30]:
def computeTF(file, uniqueWords):
    tf = []
    index = 0
    
    try :
        for sentence in file["body"]:
            tf.append(dict.fromkeys(uniqueWords, 0))
            for word in sentence:
                tf[index][word] += 1 / len(sentence)
            index += 1
    except KeyError :
        print("error")
    return tf

In [31]:
tf = computeTF(file, uniqueWords)
tf[0]

{'spez': 0.1,
 'much': 0.1,
 'longer': 0.1,
 'go': 0.1,
 'ignor': 0.1,
 'blatant': 0.1,
 'censorship': 0.1,
 'manipul': 0.1,
 'bitcoin': 0.1,
 'mod': 0.1,
 'wow': 0,
 'huge': 0,
 'surpris': 0,
 'sloppi': 0,
 'abl': 0,
 'uncov': 0,
 'vote': 0,
 'bot': 0,
 'mean': 0,
 'involv': 0,
 'hack': 0,
 'wonder': 0,
 'actual': 0,
 'what': 0,
 'crazi': 0,
 'connect': 0,
 'nullc': 0,
 'show': 0,
 'clearli': 0,
 'blockstream': 0,
 'work': 0,
 'togeth': 0,
 'entir': 0,
 'market': 0,
 'dumbfound': 0,
 'would': 0,
 'realli': 0,
 'dont': 0,
 'see': 0,
 'reddit': 0,
 'admin': 0,
 'take': 0,
 'action': 0,
 'user': 0,
 'account': 0,
 'edit': 0,
 'highli': 0,
 'recommend': 0,
 'messag': 0,
 'look': 0,
 'bleep': 0,
 'bloop': 0,
 'someon': 0,
 'link': 0,
 'thread': 0,
 'anoth': 0,
 'place': 0,
 'againstastroturf': 0,
 'evid': 0,
 'may': 0,
 'attack': 0,
 'bcore': 0,
 'berniesand': 0,
 'bgold': 0,
 'inform': 0,
 'bitcoinexpos': 0,
 'bitcoinuncensor': 0,
 'bitcoincash': 0,
 'bitcoindiscuss': 0,
 'gmaxwel': 0,
 '

### Inverse Data Frequency (IDF)

In [32]:
idfs = dict.fromkeys(uniqueWords, 0)
idfs

{'spez': 0,
 'much': 0,
 'longer': 0,
 'go': 0,
 'ignor': 0,
 'blatant': 0,
 'censorship': 0,
 'manipul': 0,
 'bitcoin': 0,
 'mod': 0,
 'wow': 0,
 'huge': 0,
 'surpris': 0,
 'sloppi': 0,
 'abl': 0,
 'uncov': 0,
 'vote': 0,
 'bot': 0,
 'mean': 0,
 'involv': 0,
 'hack': 0,
 'wonder': 0,
 'actual': 0,
 'what': 0,
 'crazi': 0,
 'connect': 0,
 'nullc': 0,
 'show': 0,
 'clearli': 0,
 'blockstream': 0,
 'work': 0,
 'togeth': 0,
 'entir': 0,
 'market': 0,
 'dumbfound': 0,
 'would': 0,
 'realli': 0,
 'dont': 0,
 'see': 0,
 'reddit': 0,
 'admin': 0,
 'take': 0,
 'action': 0,
 'user': 0,
 'account': 0,
 'edit': 0,
 'highli': 0,
 'recommend': 0,
 'messag': 0,
 'look': 0,
 'bleep': 0,
 'bloop': 0,
 'someon': 0,
 'link': 0,
 'thread': 0,
 'anoth': 0,
 'place': 0,
 'againstastroturf': 0,
 'evid': 0,
 'may': 0,
 'attack': 0,
 'bcore': 0,
 'berniesand': 0,
 'bgold': 0,
 'inform': 0,
 'bitcoinexpos': 0,
 'bitcoinuncensor': 0,
 'bitcoincash': 0,
 'bitcoindiscuss': 0,
 'gmaxwel': 0,
 'bitcoinmtl': 0,
 'bi

In [33]:
def computeIDF(idfs, numOfWords, file, len_file):
    for x, y in idfs.items():
        count = 0
        for i in range(len_file):
            try:
                for word in file["body"][i]:
                    if x == word:
                        count += 1
                        break
            except KeyError:
                i += 1
        if count != 0:
            idfs[x] = math.log(len(file) / count)
    return idfs

In [34]:
idfs = computeIDF(idfs, numOfWords, file, len_file)
idfs

{'spez': 3.248434627109745,
 'much': 2.6888188391743224,
 'longer': 3.9415818076696905,
 'go': 2.5552874465497997,
 'ignor': 4.634728988229636,
 'blatant': 3.9415818076696905,
 'censorship': 3.248434627109745,
 'manipul': 2.33214389523559,
 'bitcoin': 1.3766324502081537,
 'mod': 2.33214389523559,
 'wow': 3.248434627109745,
 'huge': 4.634728988229636,
 'surpris': 2.6888188391743224,
 'sloppi': 4.634728988229636,
 'abl': 3.5361166995615263,
 'uncov': 3.9415818076696905,
 'vote': 3.248434627109745,
 'bot': 2.5552874465497997,
 'mean': 3.5361166995615263,
 'involv': 3.5361166995615263,
 'hack': 3.0252910757955354,
 'wonder': 3.9415818076696905,
 'actual': 2.842969519001581,
 'what': 3.5361166995615263,
 'crazi': 4.634728988229636,
 'connect': 4.634728988229636,
 'nullc': 2.236833715431265,
 'show': 3.248434627109745,
 'clearli': 3.5361166995615263,
 'blockstream': 2.236833715431265,
 'work': 1.8015156441734197,
 'togeth': 3.9415818076696905,
 'entir': 3.5361166995615263,
 'market': 3.94158

### TF-IDF

In [35]:
tf[0]['spez']

0.1

In [36]:
def computeTF_IDF(tf, idfs):
    tab = []
    for i in range(len(tf)):
        dico = {}
        for mot in tf[i]:
            dico[mot] = tf[i][mot] * idfs[mot]
        tab.append(dico)
    tfidf = pd.DataFrame(tab)
    return tfidf     

In [37]:
tfidf = computeTF_IDF(tf, idfs)
tfidf

abl    absenc  absolut   account     accus    action     activ  \
0    0.000000  0.000000      0.0  0.000000  0.000000  0.000000  0.000000   
1    0.069336  0.000000      0.0  0.052722  0.000000  0.077286  0.000000   
2    0.000000  0.000000      0.0  0.000000  0.000000  0.000000  0.000000   
3    0.000000  0.000000      0.0  0.054874  0.000000  0.000000  0.000000   
4    0.000000  0.000000      0.0  0.000000  0.000000  0.000000  0.000000   
5    0.000000  0.000000      0.0  0.000000  0.000000  0.000000  0.000000   
6    0.000000  0.000000      0.0  0.000000  0.000000  0.000000  0.114249   
7    0.000000  0.000000      0.0  0.046359  0.000000  0.000000  0.000000   
8    0.000000  0.000000      0.0  0.000000  0.000000  0.000000  0.000000   
9    0.000000  0.000000      0.0  0.000000  0.091665  0.000000  0.091665   
10   0.000000  0.000000      0.0  0.141517  0.000000  0.000000  0.000000   
11   0.000000  0.000000      0.0  0.000000  0.000000  0.207452  0.000000   
12   0.000000  0.000000      0.0  0.000000  0.000000  0.000000  0.000000   
13   0.000000  0.000000      0.0  0.000000  0.000000  0.000000  0.000000   
14   0.000000  0.000000      0.0  0.000000  0.000000  0.000000  0.000000   
15   0.000000  0.000000      0.0  0.000000  0.000000  0.000000  0.000000   
16   0.000000  0.000000      0.0  0.000000  0.000000  0.000000  0.000000   
17   0.000000  0.000000      0.0  0.000000  0.000000  0.000000  0.000000   
18   0.000000  0.000000      0.0  0.000000  0.000000  0.000000  0.000000   
19   0.000000  0.000000      0.0  0.000000  0.000000  0.000000  0.000000   
20   0.000000  0.000000      0.0  0.000000  0.000000  0.000000  0.000000   
21   0.000000  0.000000      0.0  0.000000  0.000000  0.000000  0.000000   
22   0.000000  0.000000      0.0  0.000000  0.000000  0.000000  0.000000   
23   0.000000  0.000000      0.0  0.000000  0.000000  0.000000  0.000000   
24   0.000000  0.000000      0.0  0.000000  0.000000  0.000000  0.000000   
25   0.000000  0.000000      0.0  0.000000  0.000000  0.000000  0.000000   
26   0.000000  0.000000      0.0  0.000000  0.000000  0.000000  0.000000   
27   0.000000  0.000000      0.0  0.000000  0.000000  0.000000  0.000000   
28   0.000000  0.000000      0.0  0.000000  0.000000  0.000000  0.000000   
29   0.000000  0.000000      0.0  0.000000  0.000000  0.000000  0.000000   
..        ...       ...      ...       ...       ...       ...       ...   
73   0.000000  0.000000      0.0  0.000000  0.000000  0.000000  0.000000   
74   0.058448  0.000000      0.0  0.022222  0.000000  0.000000  0.000000   
75   0.000000  0.000000      0.0  0.000000  0.000000  0.000000  0.000000   
76   0.000000  0.000000      0.0  0.000000  0.000000  0.000000  0.000000   
77   0.000000  0.000000      0.0  0.000000  0.000000  0.000000  0.000000   
78   0.000000  0.000000      0.0  0.000000  0.000000  0.000000  0.000000   
79   0.000000  0.000000      0.0  0.000000  0.000000  0.000000  0.000000   
80   0.000000  0.000000      0.0  0.000000  0.000000  0.000000  0.000000   
81   0.000000  0.000000      0.0  0.000000  0.000000  0.000000  0.000000   
82   0.000000  0.000000      0.0  0.000000  0.000000  0.000000  0.000000   
83   0.000000  0.000000      0.0  0.000000  0.000000  0.000000  0.000000   
84   0.005270  0.006907      0.0  0.008014  0.005874  0.000000  0.000000   
85   0.000000  0.000000      0.0  0.000000  0.000000  0.000000  0.000000   
86   0.000000  0.000000      0.0  0.000000  0.000000  0.000000  0.000000   
87   0.000000  0.000000      0.0  0.000000  0.000000  0.000000  0.000000   
88   0.000000  0.000000      0.0  0.000000  0.000000  0.000000  0.000000   
89   0.000000  0.000000      0.0  0.000000  0.000000  0.000000  0.000000   
90   0.000000  0.000000      0.0  0.000000  0.000000  0.000000  0.000000   
91   0.000000  0.000000      0.0  0.000000  0.000000  0.000000  0.000000   
92   0.000000  0.000000      0.0  0.000000  0.000000  0.000000  0.000000   
93   0.000000  0.000000      0.0  0.000000  0.000000  0.00

### Get important words

In [38]:
columns = tfidf.columns.values
words_importance = {}
index = 0
for label in columns:
    words_importance[label] = sum(tfidf[columns[index]])
    index += 1
    
words_importance = sorted(words_importance.items(), key=operator.itemgetter(1), reverse = True)[0:50]
words_importance

[('delet', 4.131679052356995),
 ('tippr', 3.9800537803836273),
 ('bitcoin', 3.79127705447128),
 ('work', 3.1554689477020905),
 ('rbitcoin', 3.1097634889508656),
 ('reddit', 2.7970586235399275),
 ('pleas', 2.727906747427877),
 ('surpris', 2.724151041188141),
 ('post', 2.583668745941138),
 ('someon', 2.561723793694081),
 ('sticki', 2.4858215308243996),
 ('time', 2.3710321362136635),
 ('peep', 2.317364494114818),
 ('pin', 2.317364494114818),
 ('upvotedownvot', 2.317364494114818),
 ('great', 2.2704112985175637),
 ('big', 2.2523324615255373),
 ('mod', 1.9830683490288716),
 ('fuck', 1.9773642564694875),
 ('dude', 1.9707909038348452),
 ('gild', 1.8923420764274228),
 ('tell', 1.8709437165214131),
 ('true', 1.8471075040417846),
 ('nice', 1.8438322790570818),
 ('nullc', 1.8200422537056904),
 ('crosspost', 1.7737118134513608),
 ('need', 1.7570489062808259),
 ('thread', 1.7560046559669638),
 ('know', 1.6740266792651104),
 ('crimin', 1.6602244159463573),
 ('go', 1.654271307953739),
 ('come', 1.6409

### Get important documents

In [39]:
tfidf_transposed = tfidf.T
tfidf_transposed

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102
abl,0.0,0.069336,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.058448,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.005270,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0
absenc,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.006907,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0
absolut,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.000000,0.231736,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0
account,0.0,0.052722,0.000000,0.054874,0.000000,0.000000,0.000000,0.046359,0.000000,0.000000,0.141517,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.192058,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.022222,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.008014,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0
accus,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.091665,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.005874,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0
action,0.0,0.077286,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.207452,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000

In [40]:
columns = tfidf_transposed.columns.values
important_documents = {}
index = 0
for document in columns:
    important_documents[document] = sum(tfidf_transposed[columns[index]])
    index += 1
    
important_documents = sorted(important_documents.items(), key=operator.itemgetter(1), reverse = True)[0:50]
important_documents

[(12, 4.403679928042987),
 (75, 4.23636293053201),
 (98, 4.218840679893669),
 (57, 4.148251450965807),
 (97, 4.1166471486714356),
 (59, 4.078253820556571),
 (73, 4.037475831820284),
 (52, 3.9895288197449874),
 (80, 3.9419376166988846),
 (55, 3.9283730692058323),
 (9, 3.9268650904757476),
 (82, 3.9115893295333226),
 (101, 3.896204503486306),
 (46, 3.888162542717946),
 (99, 3.876939184437162),
 (92, 3.85915926790855),
 (74, 3.847159107667453),
 (77, 3.8437368902634867),
 (47, 3.8300100320125856),
 (89, 3.8300100320125856),
 (91, 3.820928667798574),
 (83, 3.815860075785216),
 (45, 3.7960473374272383),
 (22, 3.786516475173553),
 (87, 3.7816518827470262),
 (21, 3.773640518661729),
 (29, 3.7691385518102622),
 (79, 3.7388492536156086),
 (32, 3.7384394152554),
 (41, 3.7291006198982335),
 (66, 3.7286485958717286),
 (7, 3.725772251749384),
 (84, 3.722131876463352),
 (27, 3.7016628911091396),
 (34, 3.690555015340507),
 (48, 3.6812202179704316),
 (93, 3.6707767875594692),
 (65, 3.6701346574782656)

## Making a function to automize the process

In [ ]:
tab_files = os.listdir("../Data/Reddit_Data/data_history/V3_more_attributes/btc/comments")
tab_files

In [ ]:
def tf_idf_function(path, file_path):
    file = pd.read_csv(file_path.append("/").append(path))
    len_file = len(file)
    
    #Data Cleaning
    for i in range(len_file):
        file["body"][i] = re.sub(r"(\/\w\/|\w\/)", "", file["body"][i])
        file["body"][i] = re.sub(r"\\n?t?", "", file["body"][i])
        file["body"][i] = re.sub(r"[0-9]*", "", file["body"][i])
    
    file['body'] = file['body'].apply(lambda x: remove_punctuation(x))
    tokenizer = RegexpTokenizer(r'\w+')
    file['body'] = file['body'].apply(lambda x : tokenizer.tokenize(x.lower()))
    file = file[file['body'].map(len) > 2]
    for i in range(len_file):
        try :
            for word in file['body'][i]:
                if len(word) < 3:
                    file['body'][i].remove(word)
        except KeyError :
            continue
            
    file['body'] = file['body'].apply(lambda x : remove_stopwords(x))
    file['body'] = file['body'].apply(lambda x : remove_links(x))
    train_sents = treebank.tagged_sents()[:3000]
    tagger = UnigramTagger(train_sents)
    for i in range(len_file):
        try :
            for word in file["body"][i]:
                word = lemmatize(word)
        except KeyError :
            continue
    porter = PorterStemmer()
    for i in range(len_file):
        try:
            file["body"][i] = stem_sentence(file["body"][i])
        except KeyError:
            continue
    
    #Data Analysis
    uniqueWords = []

    for sentence in file["body"]:
        words = [word for word in sentence]
        for word in words:
            if word not in uniqueWords:
                uniqueWords.append(word)
                
    numOfWords = dict.fromkeys(uniqueWords, 0)
    for i in range(len_file):
        try :
            for word in file["body"][i]:
                numOfWords[word] += 1
        except KeyError:
            i += 1
            
    tf = computeTF(file, uniqueWords)
    
    idfs = dict.fromkeys(uniqueWords, 0)
    idfs = computeIDF(idfs, numOfWords, file, len_file)
    
    tfidf = computeTF_IDF(tf, idfs)
    
    #Get important words
    columns = tfidf.columns.values
    words_importance = {}
    index = 0
    for label in columns:
        words_importance[label] = sum(tfidf[columns[index]])
        index += 1
    
    words_importance = sorted(words_importance.items(), key=operator.itemgetter(1), reverse = True)[0:50]
    
    #Get important documents
    tfidf_transposed = tfidf.T
    columns = tfidf_transposed.columns.values
    important_documents = {}
    index = 0
    for document in columns:
        important_documents[document] = sum(tfidf_transposed[columns[index]])
        index += 1
    
    important_documents = sorted(important_documents.items(), key=operator.itemgetter(1), reverse = True)[0:50]